In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import numpy as np

In [ ]:
# Load the fine-tuned model and tokenizer
model_name = "/content/fine-tuned-xlm-roberta-model"
model = AutoModelForTokenClassification.from_pretrained(model_name, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def get_attention_weights(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs)
    attentions = outputs.attentions[-1].squeeze(0)  # Get the attention weights from the last layer
    return inputs, attentions
# Sample text for interpretation
sample_text = "እናት ልጇን ይዛ የተለያየ ቦታ ስትንቀሳቀስ የዱቄት ወተት የመሳሰሉትን አስፈላጊ የልጆች ምግብ ይዞ ለመንቀሳቀስ የሚረዳ 3 ፓርቲሽን ያለው አሪፍ ኮንቴነር"
inputs, attentions = get_attention_weights(model, tokenizer, sample_text)

In [ ]:
# Function to print attention weights in descending order
def print_attention_weights(attentions, inputs, tokenizer):
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
    attention_weights = attentions.mean(dim=0).detach().cpu().numpy()  # Average over heads
    token_attention_weights = []
    for i, token in enumerate(tokens):
        avg_attention = attention_weights[:, i].mean()  # Average attention for each token specifically
        token_attention_weights.append((token, avg_attention))
    # Sort by attention weight in descending order
    sorted_token_attention_weights = sorted(token_attention_weights, key=lambda item: item[1], reverse=True)
    print("Attention Weights (Descending Order):")
    for token, weight in sorted_token_attention_weights:
        print(f"Token: {token}, Weight: {weight:.4f}")
# Print the attention weights
print_attention_weights(attentions, inputs, tokenizer)